In [ ]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

# import threading, queue
import multiprocess as mp
# from multiprocessing import Process, Pipe
import signal
import os
from subprocess import check_output

from scipy.ndimage import gaussian_filter1d

In [ ]:
portAddr = "/dev/cu.usbmodem14101"
baud = 115200

In [ ]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



This just is a utility fn to extract values out of string we get from usb

In [ ]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
    extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}
    return extracted

This is the first worker. USB -> arduino_worker -> usb_reading_queue

In [ ]:

def arduino_worker(queue):
    print("worker starting...")
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                print("got", ser_bytes)
                decoded_bytes = ser_bytes.decode('ascii')
                try:
                    row = process_serial_out(decoded_bytes)
                except:
                    continue
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print_exc()
            print("Keyboard Interrupt")
            break

Prathik's code, with small modifications that might not be smart

In [ ]:
## NOTE!! MKIMMINS: this now has accel z smooth in each stride object.
class Stride:
    def __init__(self, df_walk_data, start_idx, stop_idx, accel_z_smooth):
        self.start_idx = start_idx
        self.stop_idx = stop_idx
        self.df_walk_data = df_walk_data
        self.accel_z_smooth = accel_z_smooth

        total_disp, integral1, integral2 = get_displacement_of_walk_segment (df_walk_data, start_idx, stop_idx)
        self.horizontal_length = abs(total_disp)
        self.time_duration = (df_walk_data.time[self.stop_idx] - df_walk_data.time[self.start_idx])/1000
        self.speed = self.horizontal_length/self.time_duration #m/s


def get_displacement_of_walk_segment(df_data, start_idx, end_idx):
    # first integration
    integral_0 = df_data.accel_z[start_idx:end_idx]

    # first integral
    cumsum = 0
    integral_1 = []
    for idx, elem in enumerate(integral_0):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_1.append(cumsum)
    
    # second integral
    cumsum = 0
    integral_2 = []
    for idx, elem in enumerate(integral_1):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_2.append(cumsum)
    
    # total_displacement, first_integral, second_integral
    return cumsum, integral_1, integral_2

# TODO Mkimmins : segment increased to 500!
# segment_length = 100
segment_length = 100
temp_segment_compute_limit_start = 0
temp_segment_compute_limit_end = 400
# temp_segment_compute_limit_end = len(readings)
total_walk = []
# strides = []

# Algorithm Constants
start_quiet_noise_idx = 0
end_quiet_noise_idx = 100 # 1 second sigma calibration
STRIDE_TIMEOUT = 500
num_of_stddev = 3
gaussian_filter_sigma = 5


The third worker logic. this is where we do the calculations. 
total_data -> process_calculate_strides -> total_strides
This stores each strides to total_strides. Might cut this out as non-worker just simple cell you run for ease

In [ ]:

def process_calculate_strides(total_data, total_strides):
    print("starting calculate_strides...")
    while True:
        try:
            walk_segment = total_data.get()
            print ("processing one segment chunk")
#             print("the segment: ", walk_segment)
            for k in walk_segment:total_walk.append(k) 
            df_walk = pd.DataFrame(total_walk)
            df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
            # Filter out accel_z
            accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
            # Define Baseline Noise

            accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
            accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
            upper_limit = accel_z_mean + num_of_stddev * accel_z_std
            lower_limit = accel_z_mean - num_of_stddev * accel_z_std

            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None
            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None

            for idx, data in enumerate(accel_z_smooth):
                # timeout
                if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
                    start_stride_idx = None
                    stride_under = False
                    stride_over = False
                if (data < lower_limit) and (not stride_under and not stride_over):
                    # Heal off the ground
                    stride_under = True
                    start_stride_idx = idx
                elif (data > upper_limit) and (stride_under and not stride_over):
                    stride_under = False
                    stride_over = True
                elif (data < upper_limit) and (not stride_under and stride_over):
                    stride_under = False
                    stride_over = False   
                    end_stride_idx = idx
                    curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx, accel_z_smooth)
                    print("end of reading for one stride.")
                    total_strides.put(curr_stride)
        except:
            print_exc()
            print("exception in stride processor worker")


The second worker.
It takes usb reads and chunks by segment_length, saves to total_data queue
usb_reading_queue -> stride_processor_worker -> total_data

In [ ]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data, raw_stream):
    this_segment = []
    print("stride_processor starting...")
    while True:
        #Infinitely 1. appends to this_segment, then when boundary is met,
        #appends that stride to total_data, reinitializes this_segment
        #and loops forever.
        try:
            row = raw_stream.get()
            print ("decoded: ", row)
            if len(this_segment) > segment_length:
                #put away the current stride in total data as one stride
                total_data.put(this_segment)
                print("finished writing this one stride, size: ", len(this_segment))
                #reinitialize the stride queue
                this_segment = []
            else:
                this_segment.append(row)
        except:
            print_exc()
            print("exception in stride processor worker")
            
    

In [ ]:
# overall structure:
# total_data = [stride1, stride2,,,,,,stride500]
# stride1 = [{r1}, {r2}, {r3}]

In [1]:
######### start 'main' ########

# treat this as main of code: when all the runtime things happen

Set up multithreading contexts, and initialize global queue

In [ ]:
pool = mp.Pool()
manager = mp.Manager()
usb_reading_queue = mp.SimpleQueue() # This queue holds all individual readings that are parsed.
total_data = mp.SimpleQueue() #This queue holds all chunked readings that are emptied from reading queue
total_strides = mp.SimpleQueue() #This queue drains total_data and chunks into strides with Prathik's computation



#  set up worker threads

In [ ]:
stride_process = mp.Process(target=stride_processor_worker, args=[total_data, usb_reading_queue])


In [ ]:
arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])


In [ ]:
strides_compute_proc = mp.Process(target=process_calculate_strides, args=[total_data, total_strides])

# start threads

In [ ]:
arduino_proc.start()

In [ ]:
stride_process.start()

In [ ]:
strides_compute_proc.start()

ONLY run this cell when you want to kill the processes.
if you want to inspect the intermediary queues, just kill the processes prior to that
so that the previous processes can put into queue without it being drained.

# kill threads

In [ ]:
# arduino_proc.join()
# stride_process.join()
# strides_compute_proc.join()
arduino_proc.terminate()
stride_process.terminate()
strides_compute_proc.terminate()

# SCRATCHWORK BEGINS HERE

In [ ]:
total_strides

In [ ]:
total_strides.empty()

In [ ]:
one_stride = total_strides.get()

In [ ]:
one_stride

In [ ]:
df_walk = one_stride.df_walk_data
accel_z_smooth = one_stride.accel_z_smooth

In [ ]:
fig = plt.figure(figsize=(14, 4), dpi=80)

In [ ]:
plt.plot(df_walk.time,df_walk.accel_z)
plt.plot(df_walk.time,accel_z_smooth)
plt.axhline(y=upper_limit, color='r', linestyle='-')
plt.axhline(y=lower_limit, color='r', linestyle='-')


In [ ]:

while True:
    try:
        plt.plot(df_walk.time,df_walk.accel_z)
        plt.plot(df_walk.time,accel_z_smooth)
        plt.axhline(y=upper_limit, color='r', linestyle='-')
        plt.axhline(y=lower_limit, color='r', linestyle='-')
#         plt.show()
        plt.draw()
        print ("Number of Strides: ", len(strides))
        for idx, stride in enumerate(strides):
            print ("Stride ", idx, " :") 
            print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
            print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
            print ("Stride Length (m):", stride.horizontal_length)
            print ("Stride Time Duration (sec): ", stride.time_duration)
            print ("Stride Time Duration (m/s): ", stride.speed)

        time.sleep(2)
    except:
        print("exception in plotting loop")


In [ ]:
total_data.empty()

In [ ]:
total_strides.empty()

In [ ]:

                    
                    
                    # print ((df_walk.shape))
            fig = plt.figure(figsize=(14, 4), dpi=80)
            plt.plot(df_walk.time,df_walk.accel_z)
            plt.plot(df_walk.time,accel_z_smooth)
            plt.axhline(y=upper_limit, color='r', linestyle='-')
            plt.axhline(y=lower_limit, color='r', linestyle='-')
            plt.show()
            print ("Number of Strides: ", len(strides))
            for idx, stride in enumerate(strides):
                print ("Stride ", idx, " :") 
                print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
                print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
                print ("Stride Length (m):", stride.horizontal_length)
                print ("Stride Time Duration (sec): ", stride.time_duration)
                print ("Stride Time Duration (m/s): ", stride.speed)

            time.sleep(1)

In [ ]:
total_data

In [ ]:
total_data.empty()

In [ ]:
stride_1 = total_data.get()

In [ ]:
stride_1

In [ ]:
# scratch

In [ ]:
walk_segment = total_data.get()
print("the segment: ", walk_segment)
for k in walk_segment:total_walk.append(k) 
df_walk = pd.DataFrame(total_walk)
df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
# Filter out accel_z
accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
# Define Baseline Noise

accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
upper_limit = accel_z_mean + num_of_stddev * accel_z_std
lower_limit = accel_z_mean - num_of_stddev * accel_z_std

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None


In [ ]:

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None

for idx, data in enumerate(accel_z_smooth):
    # timeout
    if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
        start_stride_idx = None
        stride_under = False
        stride_over = False
    if (data < lower_limit) and (not stride_under and not stride_over):
        # Heal off the ground
        stride_under = True
        start_stride_idx = idx
    elif (data > upper_limit) and (stride_under and not stride_over):
        stride_under = False
        stride_over = True
    elif (data < upper_limit) and (not stride_under and stride_over):
        stride_under = False
        stride_over = False   
        end_stride_idx = idx
        curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
        strides.append(curr_stride)



In [ ]:
for idx, data in enumerate(accel_z_smooth):
    print("idx: ", idx)
    print("data: ", data)

In [ ]:
strides

In [ ]:
decoded_bytes = "time=38758,accel_x= 0.03,accel_y= 0.03,accel_z= -0.39,gyro_x= 0.00,gyro_y= -0.00,gyro_z= -0.00,mag_x= 4.50,mag_y= 3.06,mag_z= -10.50,orientation_x= 359.94,orientation_y= 4.31,orientation_z= 0.44"

In [ ]:
row = process_serial_out(decoded_bytes)
print(decoded_bytes)

In [ ]:
ascii_string = decoded_bytes
ascii_string += ','
findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}


In [ ]:
extracted